# Chapter2 스파크 간단히 살펴보기
## 2.5 SparkSession

In [1]:
import findspark
findspark.init()
# jupyter와 spark 연결

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
# 스파크 세션 생성

myRange = spark.range(1000).toDF("number")
# "number" 열에 0~999 까지 1000개의 행을 가진 DataFrame생성

your 131072x1 screen size is bogus. expect trouble
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/ssuya/hadoop/spark-3.3.1/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/ssuya/hadoop/hadoop-3.3.4/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


23/02/20 19:01:54 WARN Utils: Your hostname, LAPTOP-3J0JLD9A resolves to a loopback address: 127.0.1.1; using 172.31.144.184 instead (on interface eth0)
23/02/20 19:01:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 19:01:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2.7 Transformation

In [3]:
divisBy2 = myRange.where("number % 2 = 0")
# 이 코드를 실행해도 액션을 호출하지 않으면 실제로 수행하지 않음.

## 2.8 액션

In [4]:
divisBy2.count()
# 트랜스포메이션이 count()명령으로 인해 수행되면서 500개가 결과로 나오고 출력됨.

500

# 2.10 예제


In [6]:
flightData2015 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("/user/spark/data/2015-summary.csv")

# inferschema: 데이터의 일부를 읽어서 스티마 정보를 추론
# header: 파일의 첫 행을 헤더로 지정

In [7]:
flightData2015.take(3)
# head 3개 확인

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [14]:
sorted_data = flightData2015.sort("count")
sorted_data.explain()
# 새로운 트랜스포메이션과 실행 계획
# sort는 넓은 트랜스포메이션

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#29 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#29 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [plan_id=91]
      +- FileScan csv [DEST_COUNTRY_NAME#27,ORIGIN_COUNTRY_NAME#28,count#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9900/user/spark/data/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [15]:
spark.conf.set("spark.sql.shuffle.partitions","5")
# 셔플 파티션의 개수 설정. 기본 200

In [16]:
sorted_data.take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

### 2.10.1 DataFrame과 SQL

In [17]:
flightData2015.createOrReplaceTempView("flight_data_2015")
# 임시 뷰 생성

In [19]:
sql_way = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

data_frame_way = flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .count()

In [21]:
sql_way.explain()
data_frame_way.explain()
# 동일한 실행 계획

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 5), ENSURE_REQUIREMENTS, [plan_id=113]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9900/user/spark/data/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 5), ENSURE_REQUIREMENTS, [plan_id=126]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 pa

In [23]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [26]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)
# max 함수.

[Row(max(count)=370002)]

In [27]:
max_sql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

max_sql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [28]:
from pyspark.sql.functions import desc

flightData2015\
    .groupBy("DEST_COUNTRY_NAME")\
    .sum("count")\
    .withColumnRenamed("sum(count)", "destination_total")\
    .sort(desc("destination_total"))\
    .limit(5)\
    .show()

# show에 도달 해서야 트랜스포메이션이 실행되어 결과를 도출하는 프로세스를 실행한다.

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [29]:
max_sql.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#99L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#27,destination_total#99L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[sum(count#29)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#27, 5), ENSURE_REQUIREMENTS, [plan_id=422]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#27], functions=[partial_sum(count#29)])
            +- FileScan csv [DEST_COUNTRY_NAME#27,count#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9900/user/spark/data/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


